In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

data = pd.read_csv('steam_games.csv')

In [2]:
def clean_price(price_value):
    if isinstance(price_value, (int, float)):
        return price_value
    if pd.isna(price_value):
        return 0
    if isinstance(price_value, str) and ('free' in price_value.lower() or price_value.strip() == ''):
        return 0
    if isinstance(price_value, str):
        price_str = price_value.replace('$', '').replace('€', '').replace(',', '.').strip()
        try:
            return float(price_str)
        except ValueError:
            return 0
    return 0

data['release_date'] = pd.to_datetime(data['release_date'], errors='coerce')
data['original_price'] = data['original_price'].apply(clean_price)
data['discount_price'] = data['discount_price'].fillna(data['original_price']).apply(clean_price)
data['release_year'] = data['release_date'].dt.year
data['primary_genre'] = data['genre'].str.split(',').str[0]

def categorize_price(price):
    if price < 10:
        return 0  # Günstig
    elif 10 <= price < 30:
        return 1  # Mittel
    else:
        return 2  # Teuer

data['price_category'] = data['original_price'].apply(categorize_price)

features = ['discount_price', 'release_year']
target = 'price_category'  
data_for_model = data.dropna(subset=features + [target])
X = data_for_model[features]
y = data_for_model[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
# Trainiere das Logistic Regression Modell
model = LogisticRegression(solver='liblinear', multi_class='auto', random_state=42) 
model.fit(X_train, y_train)

C:\Users\muicm\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(multi_class='auto', random_state=42, solver='liblinear')

In [4]:
print("Aussagekräftige Felder:")
if hasattr(model, 'coef_') and len(features) == model.coef_.shape[1]:  
    print("Basierend auf den Koeffizienten der Logistischen Regression:")
    for i, feature in enumerate(features):
        for j, class_name in enumerate(model.classes_):
            print(f"{feature} (Klasse {class_name}): {model.coef_[j][i]:.4f}")
else:
    print("Die Wichtigkeit der Felder konnte nicht automatisch bestimmt werden.")

print("\nKorrelationen mit der kategorisierten Zielvariablen (eingeschränkt sinnvoll):")
print(data[features + [target]].corr()[target].sort_values(ascending=False))

Aussagekräftige Felder:
Basierend auf den Koeffizienten der Logistischen Regression:
discount_price (Klasse 0): -0.0116
discount_price (Klasse 1): 0.0040
discount_price (Klasse 2): 0.0083
release_year (Klasse 0): 0.0006
release_year (Klasse 1): -0.0007
release_year (Klasse 2): -0.0016

Korrelationen mit der kategorisierten Zielvariablen (eingeschränkt sinnvoll):
price_category    1.000000
discount_price    0.018846
release_year      0.018422
Name: price_category, dtype: float64


In [5]:
# Vorhersagen auf den Testdaten
y_pred = model.predict(X_test)

# Berechne die Messmetriken
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.97      0.82       454
           1       0.18      0.02      0.04       147
           2       0.00      0.00      0.00        40

    accuracy                           0.69       641
   macro avg       0.30      0.33      0.29       641
weighted avg       0.54      0.69      0.59       641



In [6]:
# Berechne die Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)

# Berechne Sensitivität (Recall) und Spezifität (Precision) für jede Klasse
report = classification_report(y_test, y_pred, output_dict=True)
print("\nPrecision, Recall, F1-Score per Class:")
for class_label in report:
    if class_label.isdigit():
        precision = report[class_label]['precision']
        recall = report[class_label]['recall']
        f1 = report[class_label]['f1-score']
        print(f"Klasse {class_label}: Precision={precision:.2f}, Recall={recall:.2f}, F1-Score={f1:.2f}")


Confusion Matrix:
[[442  12   0]
 [143   3   1]
 [ 38   2   0]]

Precision, Recall, F1-Score per Class:
Klasse 0: Precision=0.71, Recall=0.97, F1-Score=0.82
Klasse 1: Precision=0.18, Recall=0.02, F1-Score=0.04
Klasse 2: Precision=0.00, Recall=0.00, F1-Score=0.00


### Zusammenfassung der Modellleistung und Hypothesen (50-100 Wörter)

Das Logistische Regressionsmodell zeigt die Spielpreise in drei Kategorien. Der Classification Report zeigt die Precision, Recall und F1-Scores für jede Kategorie (Kategorie 0 = günstig, Kategorie 1 = Mittel und Kategorie 2 = Teuer). Die Genauigkeit variiert zwischen den Kategorien und das heisst, dass das Modell Schwierigkeiten hat, bestimmte Preiskategorien zu unterscheiden. Vielleicht kann es sein, dass die lineare Trennbarkeit zwischen den Preiskategorien begrenzt ist oder dass zusätzliche Features wie Genre oder Publisher die Klassifikation verbessern könnten.